In [ ]:
pip install tabulate

In [9]:
import torch
import os
from transformers import AutoTokenizer
from llmcompressor.transformers import SparseAutoModelForCausalLM
import matplotlib.pyplot as plt
import numpy as np

model_stub = "neuralmagic/Meta-Llama-3.1-8B-FP8"
tokenizer = AutoTokenizer.from_pretrained(model_stub)

# Load the model
model = SparseAutoModelForCausalLM.from_pretrained(model_stub, torch_dtype=torch.float16)

weights = model.state_dict()

# # Extract weights for attention and MLP layers
# selfattn_weights = {}
attention_weights = {}
mlp_weights = {}

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.98it/s]
Decompressing model: 224it [00:27,  8.12it/s]

2024-09-24T17:08:29.853697-0400 | infer_recipe_from_model_path | INFO - Found recipe in the model_path: /Users/roshniramesh/.cache/huggingface/hub/models--neuralmagic--Meta-Llama-3.1-8B-FP8/snapshots/91994a9a9b33939a9cc0a15b8f2a1f5aafb68aef/recipe.yaml
2024-09-24T17:08:29.860002-0400 | create_instance | INFO - Loading recipe from file /Users/roshniramesh/.cache/huggingface/hub/models--neuralmagic--Meta-Llama-3.1-8B-FP8/snapshots/91994a9a9b33939a9cc0a15b8f2a1f5aafb68aef/recipe.yaml
2024-09-24T17:08:29.864081-0400 | _check_compile_recipe | INFO - Recipe compiled and 3 modifiers created



manager stage: Model structure initialized
manager stage: Model structure initialized
manager stage: Model structure initialized


2024-09-24T17:08:29.875469-0400 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 3 modifiers
2024-09-24T17:08:29.875815-0400 | initialize_recipe | INFO - Applied a staged recipe with 3 stages to the model


In [10]:
temp = {}
for name, param in weights.items():
    if any(layer in name for layer in ['.0', '15', '31']) and (
        name.endswith('self_attn.q_proj.weight') or 
        name.endswith('self_attn.k_proj.weight') or 
        name.endswith('self_attn.v_proj.weight') or 
        name.endswith('self_attn.o_proj.weight')
    ):
        # temp[name] = param.to(dtype=torch.float8_e4m3fn)
        temp[name] = param
        print(temp[name].dtype)

torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn


In [11]:
mlps = {}
if isinstance(weights, dict):
    for name, param in weights.items():
        if any(layer in name for layer in ['.0', '15', '31']) and 'mlp' in name.lower():
            mlps[name] = param
            print(mlps[name].dtype)
else:
    print("Weights is not a dictionary.")


torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn
torch.float8_e4m3fn


In [16]:
for key in mlps.keys():
    print(key)

model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.gate_proj.input_scale
model.layers.0.mlp.gate_proj.input_zero_point
model.layers.0.mlp.gate_proj.weight_scale
model.layers.0.mlp.gate_proj.weight_zero_point
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.up_proj.input_scale
model.layers.0.mlp.up_proj.input_zero_point
model.layers.0.mlp.up_proj.weight_scale
model.layers.0.mlp.up_proj.weight_zero_point
model.layers.0.mlp.down_proj.weight
model.layers.0.mlp.down_proj.input_scale
model.layers.0.mlp.down_proj.input_zero_point
model.layers.0.mlp.down_proj.weight_scale
model.layers.0.mlp.down_proj.weight_zero_point
model.layers.15.mlp.gate_proj.weight
model.layers.15.mlp.gate_proj.input_scale
model.layers.15.mlp.gate_proj.input_zero_point
model.layers.15.mlp.gate_proj.weight_scale
model.layers.15.mlp.gate_proj.weight_zero_point
model.layers.15.mlp.up_proj.weight
model.layers.15.mlp.up_proj.input_scale
model.layers.15.mlp.up_proj.input_zero_point
model.layers.15.mlp.up_proj.weight_sc

In [ ]:
print(temp)

In [ ]:
print(temp.keys())

In [ ]:
zerolayer = {name: weight for name, weight in temp.items() if any(x in name for x in ["0"])}
print(zerolayer.keys())

In [ ]:
fiflayer = {name: weight for name, weight in temp.items() if any(x in name for x in ["15"])}
print(fiflayer.keys())

In [ ]:
thirlayer = {name: weight for name, weight in temp.items() if any(x in name for x in ["31"])}
print(thirlayer.keys())

In [ ]:
colors = ['r', 'g', 'b', 'c']
plt.figure(figsize=(5, 5))

num_bins = 10
bar_width = 0.2  

x_positions = np.arange(num_bins)

for i, (layer_name, weight) in enumerate(thirlayer.items()):
    if weight.dtype == torch.float8_e4m3fn:
        weight = weight.to(torch.float32)
    weight_np = weight.detach().cpu().numpy()

    hist, bin_edges = np.histogram(weight_np.flatten(), bins=num_bins)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    plt.bar(x_positions + i * bar_width, hist, width=bar_width,
            alpha=0.5, label=layer_name, color=colors[i % len(colors)])

tick_positions = x_positions + bar_width * (len(thirlayer) - 1) / 2

plt.xticks(tick_positions, 
           [f'{bin_edges[j]:.2f} - {bin_edges[j+1]:.2f}' for j in range(num_bins)], 
           rotation=45, ha='right')

plt.title('Weight Distributions of Layer 31')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
colors = ['r', 'g', 'b', 'c']
plt.figure(figsize=(5, 5))

num_bins = 10
bar_width = 0.2  

x_positions = np.arange(num_bins)

for i, (layer_name, weight) in enumerate(fiflayer.items()):
    if weight.dtype == torch.float8_e4m3fn:
        weight = weight.to(torch.float32)
    weight_np = weight.detach().cpu().numpy()

    hist, bin_edges = np.histogram(weight_np.flatten(), bins=num_bins)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    plt.bar(x_positions + i * bar_width, hist, width=bar_width,
            alpha=0.5, label=layer_name, color=colors[i % len(colors)])

tick_positions = x_positions + bar_width * (len(fiflayer) - 1) / 2

plt.xticks(tick_positions, 
           [f'{bin_edges[j]:.2f} - {bin_edges[j+1]:.2f}' for j in range(num_bins)], 
           rotation=45, ha='right')

# Customize the plot
plt.title('Weight Distributions of Layer 15')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
colors = ['r', 'g', 'b', 'c']
plt.figure(figsize=(5, 5))

num_bins = 10
bar_width = 0.2 

x_positions = np.arange(num_bins)

for i, (layer_name, weight) in enumerate(zerolayer.items()):
    if weight.dtype == torch.float8_e4m3fn:
        weight = weight.to(torch.float32)
    weight_np = weight.detach().cpu().numpy()

    hist, bin_edges = np.histogram(weight_np.flatten(), bins=num_bins)

    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    # plt.xlim([bin_edges[0], bin_edges[-1]])
    plt.bar(x_positions + i * bar_width, hist, width=bar_width,
            alpha=0.5, label=layer_name, color=colors[i % len(colors)])

tick_positions = x_positions + bar_width * (len(zerolayer) - 1) / 2

plt.xticks(tick_positions, 
           [f'{bin_edges[j]:.2f} - {bin_edges[j+1]:.2f}' for j in range(num_bins)], 
           rotation=45, ha='right')

# Customize the plot
plt.title('Weight Distributions of Layer 0')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
print(temp['model.layers.0.self_attn.q_proj.weight'])

In [ ]:
print(temp['model.layers.0.self_attn.q_proj.weight'].shape)
print(temp['model.layers.0.self_attn.q_proj.weight'].size)



In [4]:
def extract_exp_mantissa(fp8_tensor):
    # Ensure the tensor is of float8_e4m3fn type
    if fp8_tensor.dtype != torch.float8_e4m3fn:
        raise ValueError("Input tensor must be of type torch.float8_e4m3fn")

    int_repr = fp8_tensor.view(torch.int8)
    int_repr = torch.flatten(int_repr, start_dim=0)


    # Masks for extracting mantissa and exponent
    mantissa_mask = 0b00000111  # 3 bits mask for mantissa
    exp_mask = 0b00001111       # 4 bits mask for exponent

    # Extract mantissa and exponent using bitwise operations
    mantissas = int_repr & mantissa_mask
    exponents = (int_repr >> 3) & exp_mask

    # Find the maximum mantissa and the corresponding exponent
    # torch.flatten(fp8_tensor)
    # int_repr = torch.flatten(int_repr, start_dim=0)
    # mantissas = torch.flatten(mantissas, start_dim=0)
    # exponents = torch.flatten(exponents, start_dim=0)
    
    # print(f"int_rep: {int_repr.shape}\tmantissas: {mantissas.shape}\texponents:{exponents.shape}")
    # print(mantissas)
    # print(exponents)
    max_mantissa, max_idx = torch.max(mantissas, dim=0)  # Use dim=0 for 1D result across columns
    # max_mantissa = max_mantissa.max(dim=0)
    corresponding_exponent = exponents[max_idx]


    return max_mantissa, corresponding_exponent  

# test = torch.tensor([-0.108, 1.44, 0, 0.40625], dtype=torch.float8_e4m3fn)
# mantissa, expo = extract_exp_mantissa(test)
# print(mantissa)
# print(expo)



In [ ]:
weights = temp['model.layers.0.self_attn.q_proj.weight']
weights = weights[0:16,0:16]
print(weights)

In [5]:
def max_convolution(weights, window_size=(16, 16)):
    w_height, w_width = weights.shape
    k_height, k_width = window_size

    output_shape = (w_height - k_height + 1, w_width - k_width + 1)
    max_mantissas_output = torch.zeros(output_shape, dtype=torch.int8)
    cor_exp_output = torch.zeros(output_shape, dtype=torch.int8)  # If you plan to use it later

    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            # Define the current window
            current_window = weights[i:i + k_height, j:j + k_width]
            
            # Extract mantissas from the current window
            max_mantissa, exp = extract_exp_mantissa(current_window)
            # print(max_mantissa)
            # print(max_mantissa)
            # print(exp)
            # max_val, max_ind = max_mantissa.max(dim=0) 
            
         
            max_mantissas_output[i, j] = max_mantissa.item()

            cor_exp_output[i, j] = exp.item()  
            # x = x+1

    return max_mantissas_output, cor_exp_output

# Example usage
model = 'model.layers.0.self_attn.q_proj.weight'
weights = temp[model]
# weights = weights[0:16, 0:16]

window_size = (16, 16)  
mantissas,exps = max_convolution(weights, window_size)

# Save results
mantissas_flat = mantissas.numpy().flatten()
exps_flat = exps.numpy().flatten()

with open('result.txt', 'w') as f:
    for value, exp in zip(mantissas_flat, exps_flat): 
        f.write(f"{exp} {value}\n")  

In [ ]:
import torch
from tabulate import tabulate
 
f32_type = torch.float32
bf16_type = torch.bfloat16
e4m3_type = torch.float8_e4m3fn
e5m2_type = torch.float8_e5m2

# collect finfo for each type
table = []
for dtype in [f32_type, bf16_type, e4m3_type, e5m2_type]:
    numbits = 32 if dtype == f32_type else 16 if dtype == bf16_type else 8
    info = torch.finfo(dtype)
    table.append([info.dtype, numbits, info.max, 
                  info.min, info.smallest_normal, info.eps])

headers = ['data type', 'bits', 'max', 'min', 'smallest normal', 'eps']
print(tabulate(table, headers=headers))
 
'''
Output:

data type      bits          max           min  smallest normal          eps
-------------  ----  -----------  ------------  ---------------  -----------
float32          32  3.40282e+38  -3.40282e+38      1.17549e-38  1.19209e-07
bfloat16         16  3.38953e+38  -3.38953e+38      1.17549e-38    0.0078125
float8_e4m3fn     8          448          -448         0.015625        0.125
float8_e5m2       8        57344        -57344      6.10352e-05         0.25
'''